# google画像検索結果をスクレイピングする！

今回は SeleniumというWeb ブラウザの操作を自動化するためのフレームワークを使用します

## googleDriveをマウントする

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# Seleniumを使用するのに必要なライブラリをダウンロード
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

こちらから画像を保存するフォルダを作成します

In [ ]:
!mkdir drive/MyDrive/image_data

## 実際にスクレイピングしてみる
 保存先はimage_data内にしてあります

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import time
import requests
import shutil
import base64

# ドライバーをUIなしで使用するための設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
query = input('Search word? :')
url = ("https://www.google.com/search?hl=jp&q=" + "+".join(query.split()) + "&btnG=Google+Search&tbs=0&safe=off&tbm=isch")
driver.get(url)

#適当に下までスクロールしてる--
for t in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1.5)
try:driver.find_element_by_class_name("mye4qd").click() #「検索結果をもっと表示」ってボタンを押してる
except:pass
for t in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1.5)

srcs = driver.find_elements(By.XPATH, '//img[@class="rg_i Q4LuWd"]')
try:os.mkdir('drive/MyDrive/image_data/' + query) #検索語と同じ名前のフォルダを作る、保存先
except:pass
#--

i = 0 #ファイル名に通し番号をつける用カウンタ

print("Downloading...")
for j, src in enumerate(srcs):
    if j % 50 == 0 or j == len(srcs)-1:
        print("|"+ ("■" * (20 * j // (len(srcs)-1)))+ ("　-　" * (20 - 20 * j // (len(srcs)-1)))+ "|",f"{100*j//(len(srcs)-1)}%") #ダウンロードの進捗示すやつ
    file_name = f"drive/MyDrive/image_data/{query}/{'_'.join(query.split())}_{str(i).zfill(3)}.jpg" #ファイル名とか場所とか
    src = src.get_attribute("src")
    if src != None:
#画像に変換--
        if "base64," in src:
            with open(file_name, "wb") as f:
                f.write(base64.b64decode(src.split(",")[1]))
        else:
            res = requests.get(src, stream=True)
            with open(file_name, "wb") as f:
                shutil.copyfileobj(res.raw, f)
#--
        i += 1

driver.quit() #ウィンドウを閉じる
print(f"Download is complete. {i} images are downloaded.")

Search word? :仮面ライダー
Downloading...
|　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 0%
|■　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 7%
|■■　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 14%
|■■■■　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 21%
|■■■■■　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 28%
|■■■■■■■　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 35%
|■■■■■■■■　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 42%
|■■■■■■■■■　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　　-　| 49%
|■■■■■■■■■■■　-　　-　　-　　-　　-　　-　　-　　-　　-　| 56%
|■■■■■■■■■■■■　-　　-　　-　　-　　-　　-　　-　　-　| 63%
|■■■■■■■■■■■■■■　-　　-　　-　　-　　-　　-　| 70%
|■■■■■■■■■■■■■■■　-　　-　　-　　-　　-　| 77%
|■■■■■■■■■■■■■■■■　-　　-　　-　　-　| 84%
|■■■■■■■■■■■■■■■■■■　-　　-　| 91%
|■■■■■■■■■■■■■■■■■■■　-　| 98%
|■■■■■■■■■■■■■■■■■■■■| 100%
Download is complete. 316 images are downloaded.
